In [ ]:
# Import libraries
import pandas as pd
import sqlite3

In [ ]:
# Read teams.csv file
teams = pd.read_csv('teams.csv', index_col=0)

# Connect to SQLite database
conn = sqlite3.connect('fifa_app.db')

# Save teams dataframe to Teams table in SQLite database
teams.to_sql('Teams', conn, if_exists='append', index=True, index_label='Team_ID')

# Close the connection to SQLite database
conn.close()


In [3]:
# Read the stadiums.csv file
stadiums = pd.read_csv('stadiums.csv', index_col=0)

# Connect to the database, save the dataframe to Stadiums table and close the connection
conn = sqlite3.connect('fifa_app.db')
stadiums.to_sql('Stadiums', conn, if_exists='append', index=True, index_label='Stadium_ID')
conn.close()


,Stadium_name,City,Capacity
0,Al Bayt Stadium,Al Khor City,68895
1,Lusail Stadium,Lusail City,88966
2,Ahmad Bin Ali Stadium,Umm Al Afaei,45032
3,Al Janoub Stadium,Al Wakrah,44325
4,Al Thumama Stadium,Al Thumama Stadium,44400
5,Education City Stadium,Al Rayyan,44667
6,Khalifa International Stadium,Aspire,45857
7,Stadium 974,Al Thumama Stadium,44089


In [5]:
# Read the players.csv file
players = pd.read_csv('players.csv', index_col=0)

# Replace '-' with ' ' in the Team column
players["Team"].replace("Costa-rica", "Costa Rica", inplace=True)
players["Team"].replace("Saudi-arabia", "Saudi Arabia", inplace=True)

# Normalize the names in the Name column
players['Name'] = players['Name'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

# Create a dictionary with the team names as keys and the team IDs as values
# then map the dictionary to the Team column and drop the Team column
team_dict = dict(zip(teams["Team_name"], teams.index))
players["Team_ID"] = players["Team"].map(team_dict)
players.drop("Team", axis=1, inplace=True)

# Remove the '€' and ',' from the Market_value column
players["Market_value"] = players["Market_value"].str.replace("€", "").str.replace(",", ".")

# Create a function to convert the market value to float
def convert_market_value(value):
    if value.endswith("m"):
        return float(value.rstrip("m")) * 10**6
    elif value.endswith("k"):
        return float(value.rstrip("k")) * 10**3
    else:
        return value

# Apply the function to the Market_value column and convert the column to float
players["Market_value"] = players["Market_value"].apply(convert_market_value)

# Replace the '-' values with the median value and convert the column to int
median_value = players[players["Market_value"] != "-"]["Market_value"].median()
players["Market_value"].replace("-", median_value, inplace=True)
players["Market_value"] = pd.to_numeric(players["Market_value"]).astype(int)

# Replace the values in the Better_foot column with the first letter of the value
# and fill the missing values with 'B'
players["Better_foot"] = players["Better_foot"].map({"right":"R", "left":"L", "both": "B"}).fillna("B")

# Remove the 'cm' and ',' from the Height_cm column and replace the '-' values with the median value
# then convert the column to int
players["Height_cm"] = players["Height_cm"].str.replace(",", "").str.replace("m", "")
median_value = players[players["Height_cm"] != "-"]["Height_cm"].median()
players["Height_cm"].replace("-", median_value, inplace=True)
players['Height_cm'] = players['Height_cm'].astype(int)

# Connect to the database, save the dataframe to Players table and close the connection
conn = sqlite3.connect('fifa_app.db')
players.to_sql('Players', conn, if_exists='append', index=True, index_label='Player_ID')
conn.close()


,Name,Date_of_birth,Height_cm,Position,Market_value,Better_foot,Team
0,Wojciech Szczesny,"Apr 18, 1990 (32)","1,96m",Goalkeeper,€15.00m,right,Poland
1,Bartlomiej Dragowski,"Aug 19, 1997 (24)","1,91m",Goalkeeper,€6.00m,right,Poland
2,Lukasz Skorupski,"May 5, 1991 (31)","1,87m",Goalkeeper,€4.00m,right,Poland
3,Kamil Grabara,"Jan 8, 1999 (23)","1,95m",Goalkeeper,€4.00m,right,Poland
4,Jan Bednarek,"Apr 12, 1996 (26)","1,89m",Centre-Back,€22.00m,right,Poland


In [1]:
# Read the matches.csv file
matches = pd.read_csv('matches.csv', index_col=0)

# Replace the '-' with ' ' in the Stadium column, then capitalize the first letter of each word
# and remove ' Iconic' from the end of a stadiums name
matches['Stadium'] = matches['Stadium'].str.replace("-", " ")
matches['Stadium'] = matches['Stadium'].str.title()
matches['Stadium'] = matches['Stadium'].str.replace(" Iconic", "")

# Create a dictionary with the stadium names as keys and the stadium IDs as values
# then map the dictionary to the Stadium column and drop the Stadium column
stadium_dict = dict(zip(stadiums["Stadium_name"], stadiums.index))
matches["Stadium_ID"] = matches["Stadium"].map(stadium_dict)
matches.drop("Stadium", axis=1, inplace=True)

# Create a dictionary with the team names as keys and the team IDs as values
# then map the dictionary to the Home_team and Away_team columns and drop the columns
home_team_dict = dict(zip(teams["Team_name"], teams.index))
matches["Home_team_ID"] = matches["Home_team"].map(home_team_dict)
matches.drop("Home_team", axis=1, inplace=True)

away_team_dict = dict(zip(teams["Team_name"], teams.index))
matches["Away_team_ID"] = matches["Away_team"].map(away_team_dict)
matches.drop("Away_team", axis=1, inplace=True)

# Connect to the database, save the dataframe to Matches table and close the connection
conn = sqlite3.connect('fifa_app.db')
matches.to_sql('Matches', conn, if_exists='append', index=True, index_label='Match_ID')
conn.close()


NameError: name 'pd' is not defined

In [2]:
# Read the events.csv file
events = pd.read_csv('events.csv', index_col=0)

# Create a dictionary with the player names as keys and the player IDs as values
# then map the dictionary to the player column and drop the column
player_dict = dict(zip(players["Name"], players.index))
events["Player_ID"] = events["player"].map(player_dict)
events.drop("player", axis=1, inplace=True)

# Create a dictionary with the home_team-away_team as keys and the match IDs as values
# then map the dictionary to the match column and drop the column
match_dict = {f"{row['Home_team']}-{row['Away_team']}": index for index, row in pd.read_csv('matches.csv', index_col=0).iterrows()}
events["Match_ID"] = events["match"].map(match_dict)
events.drop("match", axis=1, inplace=True)

# Connect to the database, save the dataframe to Events table and close the connection
conn = sqlite3.connect('fifa_app.db')
events.to_sql('Events', conn, if_exists='append', index=True, index_label='Event_ID')
conn.close()


NameError: name 'pd' is not defined

In [21]:
from passlib.hash import argon2

# Define the login and hash the password
login = 'admin'
password = argon2.hash("admin")

# Create a dataframe with the login and password and save it to the users dataframe
users = pd.DataFrame({'login': [login], 'password': [password]})

# Connect to the database, save the dataframe to Users table and close the connection
conn = sqlite3.connect('fifa_app.db')
users.to_sql('Users', conn, if_exists='append', index=True, index_label='User_ID')
conn.close()